# Baseline model

Just predict true if tf-idf cosin distance is closer than threshold.

This is intended for simplest end-to-end solution.

# Training-Test set setup

Dataset setup is common for all model.
Currently just put in ipynb. Please dup! (maybe factor out to .py file in the future).

### Download data of Paul from cloud storage.

https://github.com/Pawlovicky/US-patent-analysis

Data download is done outside of docker because gsutil setup is a little messy in docker.
Run following commands.

```
cd ../data
gsutil cp gs://ppr/fast_data/statistical_learning/ml-study-phys/patent/patent_applications_with_office_actions_2005-2018.zip ./
gsutil cp gs://ppr/fast_data/statistical_learning/ml-study-phys/patent/patent_grants_with_office_actions_2005-2012.zip ./
gsutil cp gs://ppr/fast_data/statistical_learning/ml-study-phys/patent/applications_with_office_actions_citing_grants_with_inner_join.zip ./
unzip applications_with_office_actions_citing_grants_with_inner_join.zip
# takes 10min or more.
unzip patent_applications_with_office_actions_2005-2018.zip
mv applicationsY.h5 patent_applications_with_office_actions_2005-2018.h5
unzip patent_grants_with_office_actions_2005-2012.zip
mv grants_n6.h5 patent_grants_with_office_actions_2005-2012.h5

```

In [1]:
import h5py
import pandas as pd
import numpy as np

In [2]:
oa_citations = pd.read_csv("../data/applications_with_office_actions_citing_grants_with_inner_join.csv").iloc[:, 1:]

/usr/local/lib/python3.5/dist-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (23,24) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
oa_citations.head()

,app_id,app_fnm,citation_pat_pgpub_id,parsed,ifw_number,action_type,action_subtype,form892,form1449,citation_in_oa,...,rejection_103,rejection_112,rejection_dp,objection,allowed_claims,cite102_gt1,cite103_gt3,cite103_eq1,cite103_max,signature_type
0,13910109,/work/data/apps/2014/ipa141204/F_1142.xml,7968444,7968444,IAFGKOGCPXXIFW4,103.0,NaN,1,1,1,...,1,0,0,1,0,0,0,0,2,0
1,13910109,/work/data/apps/2014/ipa141204/F_1142.xml,7780839,7780839,IAFGKOGCPXXIFW4,103.0,NaN,1,1,1,...,1,0,0,1,0,0,0,0,2,0
2,13910109,/work/data/apps/2014/ipa141204/F_1142.xml,7968444,7968444,IF5G7INKPXXIFW4,103.0,NaN,1,1,1,...,1,0,0,0,0,0,0,0,2,3
3,13910109,/work/data/apps/2014/ipa141204/F_1142.xml,7780839,7780839,IF5G7INKPXXIFW4,103.0,NaN,1,1,1,...,1,0,0,0,0,0,0,0,2,3
4,13910109,/work/data/apps/2014/ipa141204/F_1142.xml,7968444,7968444,IMNIOWMYPXXIFW4,103.0,NaN,1,1,1,...,1,0,0,0,0,0,0,0,2,0


In [4]:
oa_citations.shape

(1229300, 41)

In [5]:
oa_citations.app_id.unique().shape

(494374,)

# Use Rejection 102 only

In [6]:
oc_citations_103 = oa_citations[oa_citations.rejection_103 == 1]

In [7]:
oc_citations_103.shape

(1150519, 41)

In [8]:
oc_citations_102 = oa_citations[oa_citations.rejection_102 == 1]

In [9]:
oc_citations_102.shape

(574503, 41)

In [10]:
oc_citations_102.app_id.unique().shape

(268955,)

In [11]:
app_id_102s = sorted(list(set(oc_citations_102.app_id)))

In [12]:
app_id_102s[0:5]

[12000025, 12000026, 12000028, 12000033, 12000034]

## Sample 1000 training set, 1000 test set.

If app_id is uniq, we can just use dataframe.sample.
But somemodel might want to use multiple xml for the same app_id.

So keep all app_id in split phase.
Also, app_id order would be the same order as apply (maybe not, please confirm somebody!).
So keep order when split, Then shuffle so that every one reproduce split even though they change mind to use multiple xml of each app_id.

In [13]:
import random

In [14]:
random.seed(1234)

In [15]:
training_id = random.sample(app_id_102s, 1000)

In [16]:
training_id_set = set(training_id)

In [17]:
app_id_except_training = [app for app in app_id_102s if app not in training_id_set]

In [18]:
len(app_id_except_training)

267955

In [19]:
testset_id = random.sample(app_id_except_training, 1000)

In [20]:
testset_id[0:5]

[14307191, 13137006, 12741959, 12643447, 14200253]

In [21]:
len(training_id), len(testset_id)

(1000, 1000)

### Keep citation info only for training-test set.

In [60]:
target_idset = set(training_id)|set(testset_id)

In [61]:
citations_info_target = oc_citations_102[oc_citations_102.app_id.isin(target_idset)]

In [62]:
citations_info_target.shape

(4179, 41)

In [65]:
citations_info_target = citations_info_target.reset_index(drop=True)

In [66]:
citations_info_target.to_pickle("../data/citations_info_1000.df")

### Retrieve related application xmls

In [198]:
appsh5  = h5py.File('../data/patent_applications_with_office_actions_2005-2018.h5' , 'r')

In [34]:
appsh5[str(training_id[0])].value[0:50]

'<us-patent-application lang="EN" dtd-version="v4.3'

In [37]:
training_app_df = pd.DataFrame({"app_id": training_id, "xml": [appsh5[str(tid)].value for tid in training_id]})

In [38]:
training_app_df.head()

,app_id,xml
0,14222691,"<us-patent-application lang=""EN"" dtd-version=""..."
1,12515852,"<us-patent-application lang=""EN"" dtd-version=""..."
2,12033424,"<us-patent-application lang=""EN"" dtd-version=""..."
3,12402344,"<us-patent-application lang=""EN"" dtd-version=""..."
4,12155425,"<us-patent-application lang=""EN"" dtd-version=""..."


In [199]:
testset_app_df = pd.DataFrame({"app_id": testset_id, "xml": [appsh5[str(tid)].value for tid in testset_id]})

In [200]:
training_app_df.shape, testset_app_df.shape

((1000, 3), (1000, 2))

In [201]:
training_app_df.head().app_id

0    14222691
1    12515852
2    12033424
3    12402344
4    12155425
Name: app_id, dtype: int64

In [202]:
testset_app_df.iloc[1]

app_id                                             13137006
xml       <us-patent-application lang="EN" dtd-version="...
Name: 1, dtype: object

### Save application df

In [50]:
training_app_df.to_pickle("../data/training_app_1000.df")
testset_app_df.to_pickle("../data/testset_app_1000.df")

In [205]:
appsh5.close()

### Retrieve just claim. Remove all tags.

This utility function might necessary for any mode.

In [52]:
import re

In [53]:
CLAIM_PAT = re.compile(r'<claims[^>]*>(.*)</claims>',re.MULTILINE|re.DOTALL)

In [54]:
TAG_PAT = re.compile(r"<.*?>")

In [55]:
def whole_xml_to_claim_xml(whole):
    mat = CLAIM_PAT.search(whole)
    return mat.group(1)

In [56]:
def whole_xml_to_claim(whole):
    return TAG_PAT.sub(' ', whole_xml_to_claim_xml(whole))

### Retrieve grants xmls

In [57]:
grantsh5 = h5py.File('../data/patent_grants_with_office_actions_2005-2012.h5', 'r')

In [68]:
grants_target_ids = sorted(list(citations_info_target.parsed.unique()))

In [70]:
grants_target_df = pd.DataFrame({"parsed": grants_target_ids, "xml": [grantsh5[str(pid)].value for pid in grants_target_ids]})

In [71]:
grants_target_df.shape

(2524, 2)

In [72]:
grants_target_df.to_pickle("../data/grants_for_2000.df")

In [73]:
grantsh5.close()

# Model evaluation

In [67]:
citations_info_target.head()

,app_id,app_fnm,citation_pat_pgpub_id,parsed,ifw_number,action_type,action_subtype,form892,form1449,citation_in_oa,...,rejection_103,rejection_112,rejection_dp,objection,allowed_claims,cite102_gt1,cite103_gt3,cite103_eq1,cite103_max,signature_type
0,13371769,/work/data/apps/2012/ipa120607/F_2322.xml,7391316,7391316,H20LX5QGPXXIFW4,103.0,a,1,0,1,...,1,0,1,0,0,0,0,1,2,0
1,13371769,/work/data/apps/2012/ipa120607/F_2322.xml,6992580,6992580,H20LX5QGPXXIFW4,102.0,a,1,1,1,...,1,0,1,0,0,0,0,1,2,0
2,13371769,/work/data/apps/2012/ipa120607/F_2322.xml,6992580,6992580,H20LX5QGPXXIFW4,103.0,a,1,1,1,...,1,0,1,0,0,0,0,1,2,0
3,13371769,/work/data/apps/2012/ipa120607/F_2322.xml,7774833,7774833,H20LX5QGPXXIFW4,103.0,a,1,1,1,...,1,0,1,0,0,0,0,1,2,0
4,12282000,/work/data/apps/2009/ipa090312/F_1385.xml,7411209,7411209,G9LENRJ8PPOPPY5,102.0,a,0,1,1,...,1,0,0,0,0,1,0,1,1,3


In [74]:
def set_one_answer_appid(labeldf, oneappid):
    cited_patids = citations_info_target[citations_info_target.app_id == oneappid].parsed
    labeldf.loc[oneappid] = labeldf.columns.isin(cited_patids)

In [75]:
def create_label_df():
    label_df = pd.DataFrame(columns=grants_target_df.parsed.values, dtype=np.bool)
    for appid in testset_app_df.app_id:
        set_one_answer_appid(label_df, appid)
    return label_df

In [206]:
label_df = create_label_df()

In [207]:
label_df.shape

(1000, 2524)

### Cofirm label df is correct

In [208]:
testset_app_df.head()

,app_id,xml
0,14307191,"<us-patent-application lang=""EN"" dtd-version=""..."
1,13137006,"<us-patent-application lang=""EN"" dtd-version=""..."
2,12741959,"<us-patent-application lang=""EN"" dtd-version=""..."
3,12643447,"<us-patent-application lang=""EN"" dtd-version=""..."
4,14200253,"<us-patent-application lang=""EN"" dtd-version=""..."


In [210]:
label_df.head()

,6837383,6837647,6837799,6837893,6837910,6838140,6838207,6838507,6838812,6838925,...,8334161,8334431,8334887,8336128,8336158,8336789,8336964,8337193,8339697,8340894
14307191,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
13137006,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
12741959,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
12643447,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
14200253,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [212]:
label_df.loc[14307191].idxmax()

7576688

In [213]:
citations_info_target[citations_info_target["app_id"]==14307191].parsed

698    7576688
Name: parsed, dtype: int64

In [214]:
label_df.loc[14307191].loc[7576688]

True

In [215]:
label_df.loc[14307191].sum()

1

### Predict test set and print summary

In [216]:
def predict_training_set(predict_func):
    """
    predict_func(claims) return NxM of boolean. N is len(claims). M is rownum of grants_all_df.
            value indicate n claim is cite patent of m row of grants_all_df.
    """
    predictdf = pd.DataFrame(columns=grants_target_df.parsed.values, dtype=np.bool)
    res = predict_func(testset_app_df["xml"].map(whole_xml_to_claim))
    for idx, appid in enumerate(testset_app_df.app_id):
        predictdf.loc[appid] = res[idx, :]
    """
        one_res = predict_func(whole_xml_to_claim(testset_app_df[testset_app_df.app_id == appid]["xml"].iloc[0]))
        predictdf.loc[appid] = one_res
    """
    return predictdf

In [217]:
def calc_TPs(preddf, labeldf):
    return sum([sum(preddf.loc[one_appid][labeldf.loc[one_appid]]) for one_appid in testset_app_df.app_id])

def calc_FPs(preddf, labeldf):
    return sum([sum(preddf.loc[one_appid][~labeldf.loc[one_appid]]) for one_appid in testset_app_df.app_id])

def calc_TNs(preddf, labeldf):
    return sum([sum(preddf.loc[one_appid][~labeldf.loc[one_appid]] == False) for one_appid in testset_app_df.app_id])

def calc_FNs(preddf, labeldf):
    return sum([sum(preddf.loc[one_appid][labeldf.loc[one_appid]] == False) for one_appid in testset_app_df.app_id])

def calc_TFPNs(preddf, labeldf):
    return calc_TPs(preddf, labeldf), calc_FPs(preddf, labeldf), calc_TNs(preddf, labeldf), calc_FNs(preddf, labeldf)

In [218]:
def calc_summary_TFPNs(TP, FP, TN, FN):
    "return acc, prec, recall, f1."
    return pd.DataFrame(columns=["acc", "prec", "recall", "f1"], data=[[(TP+TN)/(TP+FP+TN+FN), TP/(TP+FP), TP/(TP+FN), 2*TP/(2*TP+FP+FN)]])
    
def calc_summary(preddf, labeldf):
    TP, FP, TN, FN = calc_TFPNs(preddf, labeldf)
    return calc_summary_TFPNs(TP, FP, TN, FN)

### Sample evaluation code for baseline model

predict_tfidf_model is defined below

In [219]:
pred_df = predict_training_set(predict_tfidf_model)

In [220]:
calc_summary(pred_df, label_df)

,acc,prec,recall,f1
0,0.891564,0.004106,0.901679,0.008175


# Start baseline model dependent code from here

Now common part is done.
Start model specific cells.

In [92]:
grants_target_df.head()["xml"].map(whole_xml_to_claim)

0    \n \n 1. A tool organizer for mounting to and ...
1    \n \n 1. A modular crowd and traffic control b...
2    \n \n 1. A method of aligning a putter, compri...
3    \n \n 1. Apparatus for inserting a surgical fa...
4    \n \n 1. An apparatus for gasifying a liquid o...
Name: xml, dtype: object

In [93]:
grants_target_df["claim"] = grants_target_df["xml"].map(whole_xml_to_claim)

In [94]:
grants_target_df.head()

,parsed,xml,claim
0,6837383,"<us-patent-grant lang=""EN"" dtd-version=""v40 20...",\n \n 1. A tool organizer for mounting to and ...
1,6837647,"<us-patent-grant lang=""EN"" dtd-version=""v40 20...",\n \n 1. A modular crowd and traffic control b...
2,6837799,"<us-patent-grant lang=""EN"" dtd-version=""v40 20...","\n \n 1. A method of aligning a putter, compri..."
3,6837893,"<us-patent-grant lang=""EN"" dtd-version=""v40 20...",\n \n 1. Apparatus for inserting a surgical fa...
4,6837910,"<us-patent-grant lang=""EN"" dtd-version=""v40 20...",\n \n 1. An apparatus for gasifying a liquid o...


# Convert to feature vectors and retrieve vocabulary

Doing similar things to scikit learn example  
http://scikit-learn.org/stable/auto_examples/text/document_classification_20newsgroups.html

Also, this document is helpful.  
http://scikit-learn.org/stable/tutorial/text_analytics/working_with_text_data.html

In [95]:
from sklearn.feature_extraction.text import TfidfVectorizer

## (only once) Calculate tf-idf for grants

In [96]:
random.seed(1234)

In [97]:
vectorizer = TfidfVectorizer(stop_words='english', max_df = 0.5)

In [98]:
grants_features = vectorizer.fit_transform(grants_target_df["claim"])

In [99]:
grants_features.shape

(2524, 20124)

In [100]:
vocab = vectorizer.vocabulary_

In [101]:
idfvec = vectorizer.idf_

In [102]:
len(vocab.keys())

20124

In [103]:
list(vocab.items())[0:5]

[('bindings', 2312),
 ('nucleating', 12136),
 ('bevel', 2260),
 ('cuff', 4601),
 ('decoder', 4913)]

In [104]:
len(idfvec)

20124

In [105]:
idfvec[0:5]

array([ 7.73538405,  4.86370443,  8.14084916,  8.14084916,  8.14084916])

### (only once) Save features, vocabulary, idf vector

In [106]:
import pickle

In [107]:
with open("../data/grants_target_tfidf_features.dat", "wb") as f:
    pickle.dump(grants_features, f)

In [108]:
with open("../data/grants_target_vocab_idf_dict.dat", "wb") as f:
    pickle.dump({"vocabulary": vocab, "idf": idfvec}, f)

### Load code

In [46]:
import pickle

In [47]:
with open("../data/grants_target_tfidf_features.dat", 'rb') as f:
    grants_features = pickle.load(f)

In [48]:
with open("../data/grants_target_vocab_idf_dict.dat", 'rb') as f:
    dic = pickle.load(f)
    vocab, idfvec = dic["vocabulary"], dic["idf"]

### Calculate tf-idf manually using vocabulary and idf vector, and check whether it's coinside.

In [109]:
from sklearn.feature_extraction.text import CountVectorizer

In [110]:
one_claim = grants_target_df.iloc[0]["claim"]

In [111]:
count_vec = CountVectorizer(vocabulary=vocab, stop_words="english", max_df = 0.5)

In [112]:
res = count_vec.fit_transform([one_claim])

In [113]:
res_arr = res.toarray()

In [114]:
res.shape

(1, 20124)

In [115]:
tf = res_arr[0]

In [116]:
answer = grants_features[0, :].toarray()

In [117]:
answer = answer[0]

In [118]:
def print_nonzero_index(arr, maxcount):
    count = 0

    for i, v in enumerate(arr):
        if v != 0:
            count+=1
            print(i)
            if count > maxcount:
                break

In [119]:
print_nonzero_index(answer, 5)

679
694
838
856
904
997


In [121]:
answer[997]

0.043562029896483084

In [122]:
print_nonzero_index(tf, 5)

679
694
838
856
904
997


In [123]:
sumtf = sum(tf)

In [124]:
unnormalized = [tf[i]*idfvec[i]/sumtf for i, _ in enumerate(tf)]

In [126]:
unnormalized[997]/np.linalg.norm(unnormalized)

0.043562029896483077

Try two claim for generarization

In [127]:
tfcsr = count_vec.fit_transform(grants_target_df.iloc[0:2]["claim"])

In [128]:
tf = tfcsr.toarray()

In [129]:
tf.shape

(2, 20124)

In [130]:
unnormalized = np.multiply(tf, idfvec)

In [131]:
lpnorms = np.linalg.norm(unnormalized, axis=1)

In [132]:
manual_tfidf = unnormalized/lpnorms[:, np.newaxis]

In [133]:
manual_tfidf[0, 997]

0.04356202989648307

In [134]:
all(abs(manual_tfidf[0, :] - grants_features[0].toarray()[0]) < 0.00001)

True

In [135]:
all(abs(manual_tfidf[1] - grants_features[1].toarray()[0]) < 0.00001)

True

Now make calculate tf-idf function

In [136]:
def claims_to_tfidfs(claimarr, count_vec, idfvec):
    tfcsr = count_vec.fit_transform(claimarr)
    tf = tfcsr.toarray()
    unnormalized = np.multiply(tf, idfvec)
    lpnorms = np.linalg.norm(unnormalized, axis=1)
    return unnormalized/lpnorms[:, np.newaxis]

In [137]:
manu3 = claims_to_tfidfs(grants_target_df.iloc[0:2]["claim"], count_vec, idfvec)

In [138]:
all(manu3[0] == manual_tfidf[0]), all(manu3[1] == manual_tfidf[1])

(True, True)

### It's time to calculate tfidf for training set.

In [139]:
training_app_df["claim"] = training_app_df["xml"].map(whole_xml_to_claim)

In [140]:
training_features = claims_to_tfidfs(training_app_df["claim"], count_vec, idfvec)

Calculate one cosine distance

In [141]:
one_appid = training_app_df.iloc[0].app_id

In [142]:
citations_info_target[citations_info_target.app_id == one_appid]

,app_id,app_fnm,citation_pat_pgpub_id,parsed,ifw_number,action_type,action_subtype,form892,form1449,citation_in_oa,...,rejection_103,rejection_112,rejection_dp,objection,allowed_claims,cite102_gt1,cite103_gt3,cite103_eq1,cite103_max,signature_type
1635,14222691,/work/data/apps/2014/ipa140925/F_4331.xml,8179692,8179692,I9X14IR6PXXIFW4,102.0,a,1,0,1,...,1,0,0,1,0,0,0,0,3,0
1636,14222691,/work/data/apps/2014/ipa140925/F_4331.xml,8179692,8179692,I9X14IR6PXXIFW4,103.0,NaN,1,0,1,...,1,0,0,1,0,0,0,0,3,0
1637,14222691,/work/data/apps/2014/ipa140925/F_4331.xml,8206188,8206188,I9X14IR6PXXIFW4,102.0,a,1,0,1,...,1,0,0,1,0,0,0,0,3,0
1638,14222691,/work/data/apps/2014/ipa140925/F_4331.xml,8206188,8206188,I9X14IR6PXXIFW4,103.0,NaN,1,0,1,...,1,0,0,1,0,0,0,0,3,0
1639,14222691,/work/data/apps/2014/ipa140925/F_4331.xml,8177561,8177561,I9X14IR6PXXIFW4,103.0,NaN,1,0,1,...,1,0,0,1,0,0,0,0,3,0


In [143]:
answer_patids = set(citations_info_target[citations_info_target.app_id == one_appid].parsed.astype(int))

In [144]:
answer_patids

{8177561, 8179692, 8206188}

In [146]:
type(grants_target_df.iloc[0].parsed)

numpy.int64

In [147]:
answer_idxs = grants_target_df[grants_target_df.parsed.isin(answer_patids)].index

In [148]:
answer_idxs[0]

2380

In [149]:
grants_target_df.iloc[2380].parsed

8177561

In [150]:
answer_patent_features = grants_features[answer_idxs[0], :].toarray()[0]

In [151]:
import scipy

In [152]:
scipy.spatial.distance.cdist(training_features[0, :][np.newaxis, :], grants_features[answer_idxs[0], :].toarray(), 'cosine')

array([[ 0.67340827]])

### Calculate 20 cosine distance

In [153]:
training_app_df.head()

,app_id,xml,claim
0,14222691,"<us-patent-application lang=""EN"" dtd-version=""...",\n \n 1 . A terminal comprising:\n an upper a...
1,12515852,"<us-patent-application lang=""EN"" dtd-version=""...",\n \n 1 . A method for increasing seed yield ...
2,12033424,"<us-patent-application lang=""EN"" dtd-version=""...","\n \n 1 . An image forming apparatus, compris..."
3,12402344,"<us-patent-application lang=""EN"" dtd-version=""...","\n \n 1 . A tunable polarization rotator, com..."
4,12155425,"<us-patent-application lang=""EN"" dtd-version=""...",\n \n 1 . An illumination control circuit com...


In [158]:
training_app_df[training_app_df.app_id == 12515852].index[0]

1

In [161]:
def calc_cosin_for_one_app(appid):
    answer_patids = set(citations_info_target[citations_info_target.app_id == appid].parsed.astype(int))
    answer_idxs = grants_target_df[grants_target_df.parsed.isin(answer_patids)].index
    answer_patent_features = grants_features[answer_idxs, :].toarray()
    training_features_idx = training_app_df[training_app_df.app_id == appid].index[0]
    return scipy.spatial.distance.cdist(training_features[training_features_idx, :][np.newaxis, :], answer_patent_features, 'cosine')[0]


In [162]:
calc_cosin_for_one_app(12515852)

array([ 0.4220731])

In [163]:
calc_cosin_for_one_app(12402344)

array([ 0.97559903,  0.89965045,  0.82466478,  0.63398944,  0.68337952])

In [164]:
[calc_cosin_for_one_app(appid) for appid in training_app_df[0:20].app_id]

[array([ 0.67340827,  0.76916371,  0.8174568 ]),
 array([ 0.4220731]),
 array([ 0.88221407]),
 array([ 0.97559903,  0.89965045,  0.82466478,  0.63398944,  0.68337952]),
 array([ 0.07625181]),
 array([ 0.92330727,  0.79996471,  0.92896083,  0.91987076]),
 array([ 0.90553403]),
 array([ 0.95056575]),
 array([ 0.70165577]),
 array([ 0.76213644]),
 array([ 0.99283104]),
 array([ 0.63671942]),
 array([ 0.84019419]),
 array([ 0.85744319,  0.65621906]),
 array([ 0.74489745]),
 array([ 0.39235124,  0.43900574]),
 array([ 0.58476531]),
 array([ 0.94180023,  0.97665386]),
 array([ 0.51579826,  0.67111682]),
 array([ 0.40108371])]

In [165]:
calc_cosin_for_one_app(training_app_df.iloc[5].app_id)

array([ 0.92330727,  0.79996471,  0.92896083,  0.91987076])

In [166]:
calc_cosin_for_one_app(training_app_df.iloc[5].app_id).mean() < 0.95

True

### Compare with random pair cosdistance

In [167]:
scipy.spatial.distance.cdist(training_features[0:5, :], grants_features[0:5, :].toarray(), 'cosine')

array([[ 0.96073889,  0.96715009,  0.9661195 ,  0.96547838,  0.98200139],
       [ 0.9565299 ,  0.94845901,  0.99889384,  0.95047153,  0.97128588],
       [ 0.99269734,  0.99249212,  0.99575977,  0.97310476,  0.96170229],
       [ 0.99960154,  0.99653818,  0.98222205,  0.99531781,  0.96584516],
       [ 0.85252285,  0.82698041,  0.97140084,  0.83295536,  0.90456516]])

In [168]:
training_features = claims_to_tfidfs(training_app_df["claim"], count_vec, idfvec)

In [169]:
grants_features_arr = grants_features.toarray()

In [170]:
TFIDF_MODEL_THRESHOLD=0.95
# TFIDF_MODEL_THRESHOLD=0.8

def predict_tfidf_model(claims):
    """
    return: NxM of boolean. N is len(claims). M is rownum of grants_all_df.
            value indicate n claim is cite patent of m row of grants_all_df.
    """
    features = claims_to_tfidfs(claims, count_vec, idfvec)
    dists = scipy.spatial.distance.cdist(features, grants_features_arr, 'cosine')
    return dists < TFIDF_MODEL_THRESHOLD


# Below here is manual investigation for bug fix

### Calc recall of 100

In [175]:
res = predict_tfidf_model(training_app_df[0:100]["claim"])

In [176]:
res.shape

(100, 2524)

In [177]:
all_pred_of_labeltrue = np.array([], dtype=np.bool)

In [178]:
for idx in range(0, 100):
    one_appid = training_app_df.iloc[idx].app_id
    pred_oneres = res[idx]
    label_patids = citations_info_target[citations_info_target.app_id == one_appid].parsed
    label_idxs = grants_target_df.parsed[grants_target_df.parsed.isin(label_patids)].index
    pred_of_labeltrue = pred_oneres[label_idxs]
    all_pred_of_labeltrue = np.concatenate([all_pred_of_labeltrue, pred_of_labeltrue])

In [179]:
sum(all_pred_of_labeltrue)/len(all_pred_of_labeltrue)

0.8970588235294118

### Check result by hand (seems correct)

In [180]:
training_app_df.iloc[0]

app_id                                             14222691
xml       <us-patent-application lang="EN" dtd-version="...
claim     \n \n  1 . A terminal comprising:\n an upper a...
Name: 0, dtype: object

In [181]:
pred_oneres = res[0]

In [182]:
pred_oneres.sum()

421

In [183]:
citations_info_target[citations_info_target.app_id == 14222691].parsed

1635    8179692
1636    8179692
1637    8206188
1638    8206188
1639    8177561
Name: parsed, dtype: int64

In [185]:
grants_target_df.parsed[grants_target_df.parsed == 8179692].index

Int64Index([2381], dtype='int64')

In [186]:
pred_oneres[2381]

True

### Why test set recall is so bad?

In [187]:
res = predict_tfidf_model(testset_app_df["xml"].map(whole_xml_to_claim))

In [188]:
testset_app_df.shape

(1000, 2)

In [189]:
all_pred_of_labeltrue = np.array([], dtype=np.bool)

In [190]:
for idx in range(0, testset_app_df.shape[0]):
    one_appid = testset_app_df.iloc[idx].app_id
    pred_oneres = res[idx]
    label_patids = citations_info_target[citations_info_target.app_id == one_appid].parsed
    label_idxs = grants_target_df.parsed[grants_target_df.parsed.isin(label_patids)].index
    pred_of_labeltrue = pred_oneres[label_idxs]
    all_pred_of_labeltrue = np.concatenate([all_pred_of_labeltrue, pred_of_labeltrue])

In [191]:
sum(all_pred_of_labeltrue)/len(all_pred_of_labeltrue)

0.095163806552262087

In [196]:
training_app_df.iloc[0]

app_id                                             14222691
xml       <us-patent-application lang="EN" dtd-version="...
claim     \n \n  1 . A terminal comprising:\n an upper a...
Name: 0, dtype: object

In [195]:
testset_app_df.iloc[0]

app_id                                             14222691
xml       <us-patent-application lang="EN" dtd-version="...
Name: 0, dtype: object

In [193]:
pred_oneres = res[0]

In [194]:
pred_oneres.sum()

460

In [183]:
citations_info_target[citations_info_target.app_id == 14222691].parsed

1635    8179692
1636    8179692
1637    8206188
1638    8206188
1639    8177561
Name: parsed, dtype: int64

In [185]:
grants_target_df.parsed[grants_target_df.parsed == 8179692].index

Int64Index([2381], dtype='int64')

In [186]:
pred_oneres[2381]

True

Temp code.

In [3]:
appsh5  = h5py.File('../data/patent_applications_with_office_actions_2005-2018.h5' , 'r')

In [21]:
len(set(oa_citations.app_id))

494374

In [22]:
[appid for appid, _ in  zip(set(oa_citations.app_id), range(5))]

[13631490, 14680067, 13631495, 14680072, 12582922]

In [18]:
oa_citation_app_merged = oa_citations[oa_citations.app_id.isin(appsh5.keys())]

In [20]:
oa_citation_app_merged.head()

,app_id,app_fnm,citation_pat_pgpub_id,parsed,ifw_number,action_type,action_subtype,form892,form1449,citation_in_oa,...,rejection_103,rejection_112,rejection_dp,objection,allowed_claims,cite102_gt1,cite103_gt3,cite103_eq1,cite103_max,signature_type
0,13910109,/work/data/apps/2014/ipa141204/F_1142.xml,7968444,7968444,IAFGKOGCPXXIFW4,103.0,NaN,1,1,1,...,1,0,0,1,0,0,0,0,2,0
1,13910109,/work/data/apps/2014/ipa141204/F_1142.xml,7780839,7780839,IAFGKOGCPXXIFW4,103.0,NaN,1,1,1,...,1,0,0,1,0,0,0,0,2,0
2,13910109,/work/data/apps/2014/ipa141204/F_1142.xml,7968444,7968444,IF5G7INKPXXIFW4,103.0,NaN,1,1,1,...,1,0,0,0,0,0,0,0,2,3
3,13910109,/work/data/apps/2014/ipa141204/F_1142.xml,7780839,7780839,IF5G7INKPXXIFW4,103.0,NaN,1,1,1,...,1,0,0,0,0,0,0,0,2,3
4,13910109,/work/data/apps/2014/ipa141204/F_1142.xml,7968444,7968444,IMNIOWMYPXXIFW4,103.0,NaN,1,1,1,...,1,0,0,0,0,0,0,0,2,0


In [14]:
app_key_list = set(appsh5.keys())

In [15]:
len(app_key_list)

494374

In [25]:
!ls ../data/patent_grants_with_office_actions*

../data/patent_grants_with_office_actions_2005-2012.h5
../data/patent_grants_with_office_actions_2005-2012.zip


In [27]:
appsh5.close()

In [7]:
import itertools

In [ ]:
list(itertools.islice(appsh5.keys(), 5))

In [4]:
%%time
tmp = appsh5.keys()

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 19.6 µs


In [5]:
type(tmp)

collections.abc.KeysView

In [9]:
tmpiter = iter(tmp)

In [8]:
%%time

[next(tmpiter) for _ in range(5)]

CPU times: user 2.6 s, sys: 4.86 s, total: 7.46 s
Wall time: 9min 36s


['12000001', '12000015', '12000025', '12000026', '12000028']

In [14]:
a_group_key = list(appsh5.keys())

In [15]:
len(a_group_key)

494374

In [ ]:
#print("Keys: %s" % f.keys())
a_group_key = list(f.keys())
zfile = 

In [2]:
grants_all_df = pd.read_pickle("../data/grants_2012_from2017_xmldf.dat")

In [3]:
app_all_df = pd.read_pickle("../data/app_2017_by2012_xmldf.dat")

In [29]:
training_id[0:5]

[14222691, 12515852, 12033424, 12402344, 12155425]

In [23]:
%%time
[key for key, _ in zip(appsh5.keys(), range(5))]

CPU times: user 872 ms, sys: 1.19 s, total: 2.06 s
Wall time: 2min 2s


['12000001', '12000015', '12000025', '12000026', '12000028']

In [32]:
appsh5['12000001'].value

'<us-patent-application lang="EN" dtd-version="v4.2 2006-08-23" file="US20080140920A1-20080612.XML" status="PRODUCTION" id="us-patent-application" country="US" date-produced="20080530" date-publ="20080612">\n<us-bibliographic-data-application lang="EN" country="US">\n<publication-reference>\n<document-id>\n<country>US</country>\n<doc-number>20080140920</doc-number>\n<kind>A1</kind>\n<date>20080612</date>\n</document-id>\n</publication-reference>\n<application-reference appl-type="utility">\n<document-id>\n<country>US</country>\n<doc-number>12000001</doc-number>\n<date>20071206</date>\n</document-id>\n</application-reference>\n<us-application-series-code>12</us-application-series-code>\n<priority-claims>\n<priority-claim sequence="01" kind="national">\n<country>JP</country>\n<doc-number>2006-330799</doc-number>\n<date>20061207</date>\n</priority-claim>\n<priority-claim sequence="02" kind="national">\n<country>JP</country>\n<doc-number>2007-200839</doc-number>\n<date>20070801</date>\n</p

In [25]:
dir(appsh5['12000001'])

['__array__',
 '__bool__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__nonzero__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setitem__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_d',
 '_dcpl',
 '_dxpl',
 '_e',
 '_filters',
 '_id',
 '_lapl',
 '_lcpl',
 '_local',
 'astype',
 'attrs',
 'chunks',
 'compression',
 'compression_opts',
 'dims',
 'dtype',
 'file',
 'fillvalue',
 'fletcher32',
 'id',
 'len',
 'maxshape',
 'name',
 'ndim',
 'parent',
 'read_direct',
 'ref',
 'regionref',
 'resize',
 'scaleoffset',
 'shape',
 'shuffle',
 'size',
 'value',
 'write_direct']